In [1]:
import numpy as np
import os

#from scipy.misc import imread, imresize
#from imageio import imread #using instead of imread from scipy.misc
#from scipy.misc import imresize
from skimage.io import imread #using instead of imread from scipy.misc
from skimage.transform import resize #use resize instead of imresize
import datetime
import os

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)

In [3]:
#train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
#val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

batch_size = 20 #experiment with the batch size
len(train_doc)

663

In [4]:
height = 100 # image width
width = 100 # image height
img_idx = np.arange(0, 30, 2) #create a list of image numbers you want to use for a particular video

In [5]:
classes = ['Thumbs up', 'Thumbs down','Left swipe', 'Right swipe', 'Stop']
print(classes)
num_classes = len(classes)
print(num_classes)

['Thumbs up', 'Thumbs down', 'Left swipe', 'Right swipe', 'Stop']
5


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with img_idx, y,z and normalization such that you get high accuracy.

##### Starter Generator Logic

def generator(source_path, folder_list, batch_size):
    y = 100 # image width
    z = 100 # image height

    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.arange(0,30,2) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size  # calculate the number of batches #no. of sequences/batch_size... #1 seq = 1 video... #eg: seq => 100/10 = 10
        print('Total Sequences :', num_batches * batch_size)
        x = len(img_idx)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        image = image[:, 20:140, :] # Cropping  20 px from the sides...  to make it a square image.

                    res_image = resize(image=image, output_shape=(y, z), anti_aliasing=True)

                    batch_data[folder,idx,:,:,0] = res_image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = res_image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = res_image[:,:,2] #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches

#### Generator & Support Functions

In [6]:
def process_image(source_path, folder_list, batch_size, img_ids,  img_count, height, width):
    # we iterate over the number of batches
    batch_data = np.zeros((batch_size, img_count, height, width, 3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    for folder in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_ids): #  Iterate iver the frames/images of a folder to read them in
            image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

            #crop the images and resize them. Note that the images are of 2 different shape
            #and the conv3D will throw error if the inputs in a batch have different shapes
            if image.shape[1] == 160:
                image = image[:, 20:140, :] # Cropping  20 px from the sides...  to make it a square image.

            res_image = resize(image=image, output_shape=(height, width), anti_aliasing=True)

            batch_data[folder,idx,:,:,0] = (res_image[:,:,0] / 255) #normalise and feed in the image
            batch_data[folder,idx,:,:,1] = (res_image[:,:,1] / 255) #normalise and feed in the image
            batch_data[folder,idx,:,:,2] = (res_image[:,:,2] / 255) #normalise and feed in the image

        batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [7]:
def generator(source_path, folder_list, batch_size, img_ids, height, width):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #img_ids = np.arange(0,30,2) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size  # calculate the number of batches #no. of sequences/batch_size... @ 1 seq = 1 video.. (eg: seq => 100/10 = 10)
        print('Total Sequences :', num_batches * batch_size)
        x = len(img_idx)
        for batch in range(num_batches): # we iterate over the number of batches
            yield process_image(source_path, folder_list, batch_size, img_ids, x, height, width)

        # write the code for the remaining data points which are left after full batches
        rem_batches = len(t) % batch_size
        yield process_image(source_path, folder_list, rem_batches, img_ids, x, height, width)

In [8]:
t = np.random.permutation(train_doc)
nb = len(t)//batch_size
range(nb)

range(0, 33)

## Prepare a Model

### 1. Create a Conv3D + RNN (GRU)

In [9]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.regularizers import l2
from keras import optimizers
from tensorflow.python.keras.utils.data_utils import Sequence

In [27]:
def prepare_conv3d_model():
    model = Sequential()
    model.add(Conv3D(32, (3,3,3), padding='same', input_shape=(len(img_idx), height, width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Conv3D(64, (3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Conv3D(64, (3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.20))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [28]:
conv_3d_model = prepare_conv3d_model()
optimizer = optimizers.Adam()
conv_3d_model.compile(optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
conv_3d_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 15, 100, 100, 32   2624      
                             )                                   
                                                                 
 activation_20 (Activation)  (None, 15, 100, 100, 32   0         
                             )                                   
                                                                 
 max_pooling3d_12 (MaxPooli  (None, 7, 50, 50, 32)     0         
 ng3D)                                                           
                                                                 
 conv3d_13 (Conv3D)          (None, 7, 50, 50, 64)     55360     
                                                                 
 activation_21 (Activation)  (None, 7, 50, 50, 64)     0         
                                                      

In [29]:
class LossHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.losses = []

  def on_epoch_end(self, epoch, logs={}):
    self.losses.append(logs.get('loss'))

In [30]:
history = LossHistory()
plot_data = {}

training_data_gen = generator('Project_data/train', train_doc, batch_size, img_idx, height, width)
validation_data_gen = generator('Project_data/val', val_doc, batch_size, img_idx, height, width)
epochs=3

#conv_3d_model.fit_generator(generator=training_data_gen, validation_data=validation_data_gen, epochs=epochs, callbacks=[history])
conv_3d_model.fit(x=training_data_gen, validation_data=validation_data_gen, epochs=epochs,
                  verbose=1, use_multiprocessing=True, callbacks=[history])

plot_data[lr] = history.losses

Source path =  Project_data/train ; batch size = 20
Total Sequences : 660


AttributeError: 'generator' object has no attribute 'shape'

In [31]:
len(np.arange(0,30,2))

15

In [ ]:
train_doc[:5]

In [ ]:
#"Project_data\train\WIN_20180926_17_00_22_Pro_Stop_new\WIN_20180926_17_00_22_Pro_00032.png"
#Project_data/train/WIN_20180926_17_00_22_Pro_Stop_new/WIN_20180926_17_00_22_Pro_00032.png

img = imread("Project_data/val/WIN_20180926_17_33_14_Pro_Right_Swipe_new/WIN_20180926_17_33_14_Pro_00057.png").astype(np.float32)
img.shape[1]
import matplotlib.pyplot as plt
r_img = resize(image=img[:, 20:140, :], output_shape=(100,100), anti_aliasing=True)
plt.imshow(r_img[: ,:, 0], cmap='Reds')
plt.show()
plt.imshow(r_img[: ,:, 1], cmap='Greens')
plt.show()
plt.imshow(r_img[: ,:, 2], cmap='Blues')

In [ ]:
3//2